In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error
from google.cloud import bigquery
from google.oauth2 import service_account
from math import sqrt
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
SA_FILEPATH = "/content/drive/MyDrive/Insight Estoque/service_account_unif.json"

credentials = service_account.Credentials.from_service_account_file(
    SA_FILEPATH, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

In [ ]:
project_name = 'carol-0483dfa91ba54cd4b97b.0483dfa91ba54cd4b97b1613f1171932'

In [ ]:
# query = """
#   SELECT DISTINCT tenantid, 'Sim' AS Contabil
#   FROM `{0}.accoutingforecast`
# """.format(project_name)

query = """
  SELECT tenantid, count(*) AS Contabil
  FROM `{0}.deduplicated_accoutingforecast`
  GROUP BY tenantid
""".format(project_name)

In [ ]:
df_contabil = client.query(query).to_dataframe()

In [ ]:
df_contabil

In [ ]:
# query = """
#   SELECT DISTINCT tenantid, 'Sim' AS Compras
#   FROM `{0}.demandalerts`
# """.format(project_name)

query = """
  SELECT tenantid, count(id) AS Compras
  from(
    SELECT DISTINCT tenantid, id, frequency
    FROM `{0}.deduplicated_demandgraph`
    WHERE FORMAT_DATETIME('%Y%m%d', mdmLastUpdated) = ( SELECT MAX(FORMAT_DATETIME('%Y%m%d', mdmLastUpdated)) FROM `{0}.deduplicated_demandgraph` )
    )
  GROUP BY tenantid
""".format(project_name)

In [ ]:
df_compras = client.query(query).to_dataframe()

In [ ]:
df_compras.head()

In [ ]:
# query = """
#   SELECT DISTINCT tenantid, 'Sim' AS Estoque
#   FROM `{0}.rupturealert`
# """.format(project_name)

query = """
  SELECT tenantid, count(id) AS Estoque
  FROM(
    SELECT DISTINCT tenantid, id, frequency
    FROM `{0}.deduplicated_rupturegraph`
    WHERE FORMAT_DATETIME('%Y%m%d', mdmLastUpdated) = ( SELECT MAX(FORMAT_DATETIME('%Y%m%d', mdmLastUpdated)) FROM `{0}.deduplicated_rupturegraph` )
  )
  GROUP BY tenantid
""".format(project_name)

In [ ]:
df_estoque = client.query(query).to_dataframe()

In [ ]:
df_estoque.head()

In [ ]:
# query = """
#   SELECT DISTINCT tenantid, 'Sim' AS Financeiro
#   FROM `{0}.forecastfinancial`
# """.format(project_name)

query = """
  SELECT tenantid, count(branch) AS Financeiro
  FROM(
    SELECT DISTINCT tenantid, company_group, branch, frequency
    FROM `{0}.deduplicated_forecastfinancial`
    WHERE FORMAT_DATETIME('%Y%m%d', mdmLastUpdated) = ( SELECT MAX(FORMAT_DATETIME('%Y%m%d', mdmLastUpdated)) FROM `{0}.deduplicated_forecastfinancial` )
  )
  GROUP BY tenantid
""".format(project_name)

# query = """
#   SELECT tenantid, count(branch) AS Financeiro
#   FROM(
#     SELECT DISTINCT tenantid, company_group, branch, frequency
#     FROM `{0}.deduplicated_financialalert`
#   )
#   GROUP BY tenantid
# """.format(project_name)

In [ ]:
df_financeiro = client.query(query).to_dataframe()

In [ ]:
df_financeiro.head()

In [ ]:
df_financeiro['Financeiro'].sum()

In [ ]:
df = df_financeiro.merge(df_estoque, how='outer', on=['tenantid'])

In [ ]:
df = df.merge(df_compras, how='outer', on=['tenantid'])

In [ ]:
df = df.merge(df_contabil, how='outer', on=['tenantid'])

In [ ]:
df.shape

In [ ]:
# df.fillna('Não', inplace=True)
df.fillna(0, inplace=True)

In [ ]:
df.loc[df.Financeiro > 0, "Financeiro"] = 1
df.loc[df.Estoque > 0, "Estoque"] = 1
df.loc[df.Compras > 0, "Compras"] = 1
df.loc[df.Contabil > 0, "Contabil"] = 1

In [ ]:
df.head()

In [ ]:
df.to_excel('/content/drive/MyDrive/Insight Estoque/tenants_insights.xlsx')

### Clientes Piloto

In [ ]:
# df_pilotos = pd.read_excel('/content/drive/MyDrive/Insight Estoque/pilotos.xlsx')

In [ ]:
# df_pilotos

In [ ]:
# df['Contabil'].sum()